In [6]:
#import sys
#!{sys.executable} -m pip install torch gym_conservation "ray[rllib, tune]"

In [7]:
import torch
import gym
import gym_conservation
import gym_fishing
from ray import tune
from ray.rllib import agents
from ray.rllib.agents import ppo, impala
torch.cuda.device_count()



1

In [8]:
# Custom environment creator utility
def env_creator(env_name):
    if env_name == 'fishing-v1':
        from gym_fishing.envs import FishingCtsEnv as env
    elif env_name == "conservation-v6":
        from gym_conservation.envs import NonStationaryV6 as env
    elif env_name == "fishing-v0":
        from gym_fishing.envs import FishingEnv as env
    else:
        raise NotImplementedError
    return env


In [9]:
# Use custom env creator instead of gym.make() to create callable env
# Then register env with label in ray tune.
env = env_creator('conservation-v6')
tune.register_env('conservation-v6', lambda config: env())
env1 = env_creator("fishing-v0")
tune.register_env('fishing-v0', lambda config: env1())
env1 = env_creator("fishing-v1")
tune.register_env('fishing-v1', lambda config: env1())

In [10]:
## minimal example
#import gym, ray
#from ray.rllib.agents.ppo import PPOTrainer
#trainer = PPOTrainer(env="fishing-v1", config={})
#trainer.train()

In [12]:
# Configure the algorithm.
config = {
    # Environment (RLlib understands openAI gym registered strings).
    "env": "conservation-v6",
    # Use 2 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 8,
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "torch",
    "num_gpus": 1,
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    "model": {
         "use_lstm": True,
#        "fcnet_hiddens": [128, 128],
#        "fcnet_activation": "relu",
    },
    # Set up a separate evaluation worker set for the
    # `trainer.evaluate()` call after training (see below).
    "evaluation_num_workers": 1,
    # Only for evaluation runs, render the env.
    "evaluation_config": {
        "render_env": False,
    }
}

# Create our RLlib Trainer.
trainer = impala.ImpalaTrainer(config=config)



2022-01-27 23:37:47,879	WARNING trainer.py:1948 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Trainer.train()`. Instead, you will have to call `Trainer.evaluate()` manually in order to trigger an evaluation run.
2022-01-27 23:37:47,880	WARNING ppo.py:151 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=6 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 666.
2022-01-27 23:37:49,263	WARNING services.py:1826 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67084288 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to s

In [13]:
for _ in range(500):
    trainer.train()

In [17]:
trainer.evaluate()


{'evaluation': {'episode_reward_max': 54.26525394784858,
  'episode_reward_min': 39.04949102949177,
  'episode_reward_mean': 47.02623127274943,
  'episode_len_mean': 487.8,
  'episode_media': {},
  'episodes_this_iter': 10,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_mean': {},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [50.880918718155684,
    45.3586469829504,
    46.00949622415576,
    47.330491343408504,
    43.318821816283204,
    51.78151185551697,
    54.26525394784858,
    50.00327623484258,
    42.26440457484087,
    39.04949102949177],
   'episode_lengths': [476, 501, 501, 394, 501, 501, 501, 501, 501, 501]},
  'sampler_perf': {'mean_raw_obs_processing_ms': 0.0947874866320623,
   'mean_inference_ms': 1.01094974995539,
   'mean_action_processing_ms': 0.10084617233198181,
   'mean_env_wait_ms': 0.2282006399972098,
   'mean_env_render_ms': 0.0},
  'off_policy_estimator': {}}}